In [1]:
import pandas as pd
import plotly.graph_objects as go
import os
import requests

In [2]:
def token():
    if os.path.exists("token.txt"):
        with open("token.txt", "r") as file:
            return file.read()
    return print("VelibFlow/token.txt does not exist, could get one here :  https://plotly.com/.")

In [3]:
def make_request(path: str):
    return requests.get(path)


def get_availablity_bikes():
    # Requêtes API Velib
    stations_information_response = make_request('https://velib-metropole-opendata.smovengo.cloud/opendata/Velib_Metropole/station_information.json')
    stations_status_response = make_request('https://velib-metropole-opendata.smovengo.cloud/opendata/Velib_Metropole/station_status.json')
    stations_information = pd.DataFrame(stations_information_response.json()['data']['stations'])
    stations_status = pd.DataFrame(stations_status_response.json()['data']['stations'])

    # Fonction pour extraire les valeurs entières des dictionnaires
    def extract_integer(dictionary_list, key):
        for dictionary in dictionary_list:
            if key in dictionary:
                return dictionary[key]
        return None

    # Extraction des valeurs entières pour les colonnes "mechanical" et "ebike"
    stations_status['mechanical'] = stations_status['num_bikes_available_types'].apply(lambda x:
                                                                                       extract_integer(x, 'mechanical'))
    stations_status['ebike'] = stations_status['num_bikes_available_types'].apply(lambda x: extract_integer(x, 'ebike'))

    # Suppression de la colonne d'origine
    stations_status.drop('num_bikes_available_types', axis=1, inplace=True)

    # Conversion en entier des codes station StationCode
    stations_status['stationCode'] = stations_status['stationCode'].astype(str)
    stations_information['stationCode'] = stations_information['stationCode'].astype(str)

    # Merge des deux tables
    stations = stations_information.merge(stations_status, on=['station_id', 'stationCode'])

    # Conversion d'un timestamp en datetime
    stations['last_reported'] = pd.to_datetime(stations['last_reported'], unit='s')

    # Creation de KPIs
    stations['availableBikesRate'] = (stations['numBikesAvailable'] / stations['capacity']) * 100
    stations['availableDocksRate'] = (stations['numDocksAvailable'] / stations['capacity']) * 100
    stations['availableMechanicalBikesRate'] = (stations['mechanical'] / stations['capacity']) * 100
    stations['availableElectricRate'] = (stations['ebike'] / stations['capacity']) * 100

    # Remplacement des NaN par 0 dans les colonnes KPIs
    stations['availableBikesRate'] = stations['availableBikesRate'].fillna(0)
    stations['availableDocksRate'] = stations['availableDocksRate'].fillna(0)
    stations['availableMechanicalBikesRate'] = stations['availableMechanicalBikesRate'].fillna(0)
    stations['availableElectricRate'] = stations['availableElectricRate'].fillna(0)

    # Arrondir les KPIs au dixième
    stations['availableBikesRate'] = stations['availableBikesRate'].round(2)
    stations['availableDocksRate'] = stations['availableDocksRate'].round(2)
    stations['availableMechanicalBikesRate'] = stations['availableMechanicalBikesRate'].round(2)
    stations['availableElectricRate'] = stations['availableElectricRate'].round(2)

    return stations


In [4]:
def fetch():
    velib_data = get_availablity_bikes()
    velib_data.to_csv('update_data.csv', header=False, mode='a', date_format='%Y-%m-%d %H:%M:%S')

In [7]:
def show_map(dataset: pd.DataFrame = get_availablity_bikes(), color_by: str = 'availableBikesRate'):
    px.set_mapbox_access_token(token())
    fig = px.scatter_mapbox(dataset, lat="lat", lon="lon", color=color_by,
                            color_continuous_scale=px.colors.sequential.Brwnyl, size_max=15, zoom=9.5,
                            range_color=(0, 100))
    fig.show()

In [7]:
dataset = pd.DataFrame(get_availablity_bikes())
color_by = 'availableBikesRate'


fig = px.scatter(dataset, x="lon", y="lat", color=color_by,
                 color_continuous_scale=px.colors.sequential.Brwnyl, size_max=15,
                 range_color=(0, 100),
                 hover_data=['name', 'num_bikes_available', 'num_docks_available'])

fig.update_layout(title='Bike Availability Rates in Paris',
                  coloraxis_colorbar=dict(title='Available Bikes Rate (%)'))

fig.show()

In [6]:
import plotly.graph_objects as go

dataset = pd.DataFrame(get_availablity_bikes())
color_by = 'availableBikesRate'

color_by = 'availableBikesRate'

scatter = go.Scattergeo(
    lon=dataset['lon'],
    lat=dataset['lat'],
    mode='markers',
    marker=dict(
        size=15,
        color=dataset[color_by],
        colorscale='Brwnyl',
        cmin=0,
        cmax=100,
        colorbar=dict(title='Available Bikes Rate (%)')
    ),
    hoverinfo='text',
    text=dataset[['name', 'num_bikes_available', 'num_docks_available']],
)

layout = go.Layout(
    title='Bike Availability Rates in Paris',
    geo=dict(
        center=dict(lat=48.8566, lon=2.3522),  # Coordinates for Paris
        projection_scale=5,  # Adjust the scale as needed
        showland=True,
    ),
)

fig = go.Figure(data=[scatter], layout=layout)
fig.show()

sk.eyJ1IjoibHluZGFzdGFya3VzIiwiYSI6ImNscXdmcWZ1ejAyeXAyd3BocjA2c3plZm4ifQ.eCRqtq5DGHoCwJBQOSTGYQ
